# Capstone project - HealthCare Fraud Detection : Logistic Regression

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from cp_clean_helper import show_values
plt.style.use('ggplot')
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 100)

### # Loading datasets and looking at shapes:-

In [ ]:
# Loading datasets

train_df = pd.read_csv("clean_train.csv",index_col=0)
test_df = pd.read_csv("clean_test.csv",index_col=0)

# Looking at dataset shapes

print('Shape of train data :',train_df.shape)
print('Shape of test data :',test_df.shape)

### # Looking at first few rows for each:

In [ ]:
train_df.head(5)

In [ ]:
test_df.head(5)